In [128]:
# import libraries (you may add additional imports but you may not have to)
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
import matplotlib.pyplot as plt

In [129]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/books/book-crossings.zip

!unzip book-crossings.zip

books_filename = 'BX-Books.csv'
ratings_filename = 'BX-Book-Ratings.csv'

--2024-03-17 10:23:44--  https://cdn.freecodecamp.org/project-data/books/book-crossings.zip
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 172.67.70.149, 104.26.2.33, 104.26.3.33, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|172.67.70.149|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 26085508 (25M) [application/zip]
Saving to: ‘book-crossings.zip.9’

book-crossings.zip. 100%[===================>]  24.88M  62.1MB/s    in 0.4s    

2024-03-17 10:23:44 (62.1 MB/s) - ‘book-crossings.zip.9’ saved [26085508/26085508]

Archive:  book-crossings.zip
replace BX-Book-Ratings.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: N


In [130]:
df_books = pd.read_csv(
    books_filename,
    encoding="ISO-8859-1",
    sep=";",
    header=0,
    names=['isbn', 'title', 'author'],
    usecols=['isbn', 'title', 'author'],
    dtype={'isbn': 'str', 'title': 'str', 'author': 'str'}
)

df_ratings = pd.read_csv(
    ratings_filename,
    encoding="ISO-8859-1",
    sep=";",
    header=0,
    names=['user', 'isbn', 'rating'],
    usecols=['user', 'isbn', 'rating'],
    dtype={'user': 'int32', 'isbn': 'str', 'rating': 'float32'}
)



In [131]:
user_counts = df_ratings['user'].value_counts()
df_ratings = df_ratings[df_ratings['user'].isin(user_counts[user_counts >= 10].index)]

book_counts = df_ratings['isbn'].value_counts()
df_ratings = df_ratings[df_ratings['isbn'].isin(book_counts[book_counts >= 200].index)]



In [132]:
# merge dataframes
df = pd.merge(df_ratings, df_books, on='isbn', how='inner')



In [133]:
# pivot table
pivot_df = df.pivot_table(index='title', columns='user', values='rating').fillna(0)



In [134]:
# create sparse matrix
sparse_matrix = csr_matrix(pivot_df.values)



In [135]:
# instantiate and fit nearest neighbors model
model_knn = NearestNeighbors(metric='cosine', algorithm='brute')
model_knn.fit(sparse_matrix)



NearestNeighbors(algorithm='brute', metric='cosine')

In [136]:
# function to get recommendations
def get_recommends(book_title):
    distances, indices = model_knn.kneighbors(pivot_df.loc[book_title].values.reshape(1, -1), n_neighbors=6)
    recommended_books = []
    for i in range(1, len(distances.flatten())):
        recommended_books.append([pivot_df.index[indices.flatten()[i]], distances.flatten()[i]])
    return [book_title, recommended_books]

In [137]:
# Test the get_recommends function
recommendations = get_recommends("The Queen of the Damned (Vampire Chronicles (Paperback))")
print(recommendations)

['The Queen of the Damned (Vampire Chronicles (Paperback))', [['The Vampire Lestat (Vampire Chronicles, Book II)', 0.52458334], ['Interview with the Vampire', 0.6495025], ['The Witching Hour (Lives of the Mayfair Witches)', 0.84850085], ['The Catcher in the Rye', 0.9208695], ['A Time to Kill', 0.93499404]]]
